# Agenda

- Generator functions
- Generator comprehensions
- Concurrency
    - Threads
    - Multiprocessing
    - `asyncio`

In [1]:
def myfunc(x, y):
    for one_number in range(x, y):
        yield one_number

In [2]:
myfunc(10, 20)

<generator object myfunc at 0x10ce49700>

In [3]:
for one_item in myfunc(10, 20):
    print(one_item)

10
11
12
13
14
15
16
17
18
19


In [4]:
def evenrange(start, finish):
    for one_number in range(start, finish):
        if one_number % 2:
            continue
        yield one_number

In [5]:
for one_item in evenrange(10, 20):
    print(one_item)

10
12
14
16
18


# Things to remember about generators

- `yield` means: return the value (in the iteration/loop), and go to sleep right after
- if we use `return` without an argument, we effectively raise `StopIteration`
- Also when the function exits normally/naturally, we effectively raise `StopIteration`
- Don't raise `StopIteration` in a generator function! Let the system do it for you
- Local variables retain their values between invocations of `yield`

In [6]:
def myfunc():
    return f'Hello, world!'

In [7]:
def mygen():
    yield f'Hello, world!'

In [9]:
myfunc()

'Hello, world!'

In [10]:
mygen()

<generator object mygen at 0x10cebfd70>

In [11]:
type(myfunc)

function

In [12]:
type(mygen)

function

In [13]:
myfunc.__code__.co_consts

(None, 'Hello, world!')

In [14]:
myfunc.__code__.co_flags

3

In [16]:
bin(myfunc.__code__.co_flags)

'0b11'

In [17]:
bin(mygen.__code__.co_flags)

'0b100011'

In [18]:
import dis

In [19]:
dis.show_code(myfunc)

Name:              myfunc
Filename:          /var/folders/rr/0mnyyv811fs5vyp22gf4fxk00000gn/T/ipykernel_76740/2896535200.py
Argument count:    0
Positional-only arguments: 0
Kw-only arguments: 0
Number of locals:  0
Stack size:        1
Flags:             OPTIMIZED, NEWLOCALS
Constants:
   0: None
   1: 'Hello, world!'


In [20]:
dis.show_code(mygen)

Name:              mygen
Filename:          /var/folders/rr/0mnyyv811fs5vyp22gf4fxk00000gn/T/ipykernel_76740/3368138942.py
Argument count:    0
Positional-only arguments: 0
Kw-only arguments: 0
Number of locals:  0
Stack size:        1
Flags:             OPTIMIZED, NEWLOCALS, GENERATOR
Constants:
   0: None
   1: 'Hello, world!'


In [21]:
def myfunc():
    return 1
    return 2
    return 3

In [22]:
dis.dis(myfunc)

  1           0 RESUME                   0

  2           2 LOAD_CONST               1 (1)
              4 RETURN_VALUE


In [23]:
def myfunc():
    if False:
        yield 'Hello'
    

In [24]:
dis.show_code(myfunc)

Name:              myfunc
Filename:          /var/folders/rr/0mnyyv811fs5vyp22gf4fxk00000gn/T/ipykernel_76740/1059155119.py
Argument count:    0
Positional-only arguments: 0
Kw-only arguments: 0
Number of locals:  0
Stack size:        1
Flags:             OPTIMIZED, NEWLOCALS, GENERATOR
Constants:
   0: None


In [25]:
g1 = evenrange(10, 20)
g2 = evenrange(10, 20)
g3 = evenrange(10, 20)


In [26]:
next(g1)

10

In [27]:
next(g1)



12

In [28]:
next(g1)

14

In [29]:
g1.gi_frame.f_lineno

5

In [30]:
g2.gi_frame.f_lineno

1

# Exercise: `read_n`

When we iterate over a file, each iteration gives us the next line, up to and including `\n`. Sometimes, though, we want to read a file in chunks of 2, 5, or 10 lines.

1. Write `read_n`, that takes two arguments:
    - The name of a file
    - The number of lines we want to get back in each iteration
2. With each iteration, we should get that number of lines back, as a single string.
3. The final iteration can give us fewer lines.

Example:

```python
for one_chunk in read_n('/etc/passwd', 5):
     print(one_chunk) 
```

In [31]:
def read_n(filename, n):
    with open(filename) as f:
        while True:
            output = f.readline()
    
            if output:
                yield output
            else:
                return

In [32]:
for one_chunk in read_n('/etc/passwd', 5):
    print(one_chunk)

##

# User Database

# 

# Note that this file is consulted directly only when the system is running

# in single-user mode.  At other times this information is provided by

# Open Directory.

#

# See the opendirectoryd(8) man page for additional information about

# Open Directory.

##

nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false

root:*:0:0:System Administrator:/var/root:/bin/sh

daemon:*:1:1:System Services:/var/root:/usr/bin/false

_uucp:*:4:4:Unix to Unix Copy Protocol:/var/spool/uucp:/usr/sbin/uucico

_taskgated:*:13:13:Task Gate Daemon:/var/empty:/usr/bin/false

_networkd:*:24:24:Network Services:/var/networkd:/usr/bin/false

_installassistant:*:25:25:Install Assistant:/var/empty:/usr/bin/false

_lp:*:26:26:Printing Services:/var/spool/cups:/usr/bin/false

_postfix:*:27:27:Postfix Mail Server:/var/spool/postfix:/usr/bin/false

_scsd:*:31:31:Service Configuration Service:/var/empty:/usr/bin/false

_ces:*:32:32:Certificate Enrollment Service:/var/empty:/usr/bin/fal

In [33]:
# let's take n into account

def read_n(filename, n):
    with open(filename) as f:
        while True:
            output_lines = []
            
            for i in range(n):
                output_lines.append(f.readline())

            output = ''.join(output_lines)
                
            if output:
                yield output
            else:
                return

In [34]:
for one_chunk in read_n('/etc/passwd', 5):
    print(one_chunk)

##
# User Database
# 
# Note that this file is consulted directly only when the system is running
# in single-user mode.  At other times this information is provided by

# Open Directory.
#
# See the opendirectoryd(8) man page for additional information about
# Open Directory.
##

nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false
root:*:0:0:System Administrator:/var/root:/bin/sh
daemon:*:1:1:System Services:/var/root:/usr/bin/false
_uucp:*:4:4:Unix to Unix Copy Protocol:/var/spool/uucp:/usr/sbin/uucico
_taskgated:*:13:13:Task Gate Daemon:/var/empty:/usr/bin/false

_networkd:*:24:24:Network Services:/var/networkd:/usr/bin/false
_installassistant:*:25:25:Install Assistant:/var/empty:/usr/bin/false
_lp:*:26:26:Printing Services:/var/spool/cups:/usr/bin/false
_postfix:*:27:27:Postfix Mail Server:/var/spool/postfix:/usr/bin/false
_scsd:*:31:31:Service Configuration Service:/var/empty:/usr/bin/false

_ces:*:32:32:Certificate Enrollment Service:/var/empty:/usr/bin/false
_appstore:*:3

In [35]:
# let's take n into account

def read_n(filename, n):
    with open(filename) as f:
        while True:
            output = ''.join([f.readline()
                              for i in range(n)])
                
            if output:
                yield output
            else:
                return

for one_chunk in read_n('/etc/passwd', 7):
    print(one_chunk)

##
# User Database
# 
# Note that this file is consulted directly only when the system is running
# in single-user mode.  At other times this information is provided by
# Open Directory.
#

# See the opendirectoryd(8) man page for additional information about
# Open Directory.
##
nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false
root:*:0:0:System Administrator:/var/root:/bin/sh
daemon:*:1:1:System Services:/var/root:/usr/bin/false
_uucp:*:4:4:Unix to Unix Copy Protocol:/var/spool/uucp:/usr/sbin/uucico

_taskgated:*:13:13:Task Gate Daemon:/var/empty:/usr/bin/false
_networkd:*:24:24:Network Services:/var/networkd:/usr/bin/false
_installassistant:*:25:25:Install Assistant:/var/empty:/usr/bin/false
_lp:*:26:26:Printing Services:/var/spool/cups:/usr/bin/false
_postfix:*:27:27:Postfix Mail Server:/var/spool/postfix:/usr/bin/false
_scsd:*:31:31:Service Configuration Service:/var/empty:/usr/bin/false
_ces:*:32:32:Certificate Enrollment Service:/var/empty:/usr/bin/false

_appstore:*:33

# Generator comprehensions

In [36]:
# list comprehension

[x*x
 for x in range(-10, 10)]

[100, 81, 64, 49, 36, 25, 16, 9, 4, 1, 0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [37]:
# set comprehension

{x*x
 for x in range(-10, 10)}

{0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100}

In [38]:
# dict comprehension

{x : x*x
 for x in range(-10, 10)}

{-10: 100,
 -9: 81,
 -8: 64,
 -7: 49,
 -6: 36,
 -5: 25,
 -4: 16,
 -3: 9,
 -2: 4,
 -1: 1,
 0: 0,
 1: 1,
 2: 4,
 3: 9,
 4: 16,
 5: 25,
 6: 36,
 7: 49,
 8: 64,
 9: 81}

In [41]:
# generator expression / generator comprehension

g = (x*x
 for x in range(-10, 10))
g

<generator object <genexpr> at 0x10cd871d0>

In [42]:
for one_item in g:
    print(one_item)

100
81
64
49
36
25
16
9
4
1
0
1
4
9
16
25
36
49
64
81


In [43]:
mylist = [10, 20, 30]

'*'.join(mylist)   # this won't work -- only can pass a list of strings

TypeError: sequence item 0: expected str instance, int found

In [44]:
# we can use a list comprehension here

'*'.join([str(one_item)
          for one_item in mylist])

'10*20*30'

In [45]:
# we can also use a generator comprehension

'*'.join(   (str(one_item)
            for one_item in mylist)    )

'10*20*30'

In [46]:
# what about this:
# we can remove the inner parentheses and it works fine!

'*'.join(   str(one_item)
            for one_item in mylist    )

'10*20*30'

In [52]:
g = (   print(one_item)
    for one_item in mylist    )

In [51]:
print(next(g))

20
None


In [53]:
list(g)

10
20
30


[None, None, None]

In [54]:
numbers = {x*x
 for x in range(-10, 10)}

In [55]:
print(numbers)  # __repr__ 

{64, 1, 0, 100, 36, 4, 9, 16, 81, 49, 25}


In [56]:
numbers   # __str__

{0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100}

In [57]:
d = {}
d['a'] = 100    

In [59]:
hash('a') % 8

4

# Dict talk by Raymond Hettinger
https://www.youtube.com/watch?v=p33CVV29OG8

# Exercise: Only vowels

Write a generator expression that returns the vowels (a, e, i, o, and u) from a file

Example:

```python
for one_item in g:
     print(one_item)  
```

or

```python
list(g)
```


In [65]:
filename = '/etc/passwd' 

g = ( one_character
  for one_line in open(filename)
  for one_character in one_line 
  if one_character.lower() in 'aeiou')

for one_item in g:
     print(one_item, end=' ')  


U e a a a e o e a i i e i o u e i e o e e e i u i i i e u e o e A o e i e i i o a i o i o i e O e i e o e e e o e i e o a a e o a i i o a i o a i o a o u O e i e o o o U i i e e U e a e u i a e o o e A i i a o a o o i a e o e e i e a o o u i a e u u U i o U i o o o o a o o u u u i u u i o a a e a a e a e o a e u i a e e o e o e i e a e o u i a e i a a i a I a A i a a e u i a e i i e i e a o o u u i a e o i o i a i e e a o o o i u i a e e i e o i u a i o e i e a e u i a e e e i i a e E o e e i e a e u i a e a o e a A o e e i e a a o e u i a e a A a u a e u i a e a e e e A e E e a e o a e u i a e e o e o e i e a e o a e o u i a e e o e e o e o u e a i o a e u i a e a o e a e a e u i a e e o e e o e a e u i a e a A e e o e e o a e u i a e o i e e e e i a e e e u i a e e A e E e U e a e u i a e e e a e u i a e e e a e u i a e e e a e u i a e i i e e e a a i o a e u i a e u i i e e a i e e a e u i a e u u A i i a o a i a u i a e a i a a i a i e e a e u i a e a e e A i a i o e e a e u i a e 

# Next up

1. Threads
2. Multiprocessing

Resume at :25

GIL -- global interpreter lock

only one thread executes at a time



# Exercise: File vowels

1. Define a function, `count_vowels`, that takes a filename as an input. It uses `q.put` to return a tuple with two elements:
    - the filename
    - a dictionary with keys a, e, i, o, and u as an output. The values will be integers, the number of times each vowel appears in a file.
3. Use threads to start this function once per file in a directory (or a list of files that you provide). You can use `os.listdir` or `glob.glob`.
4. When all of the threads are done, go through the queue and print all of the filenames and the reports they've made.
5. If a file doesn't work (binary, no permission, etc.), then just ignore it.